In [123]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math

import nacl.bindings as nb
import pickle 
import sys
import random
import pandas as pd
import numpy as np
import math

In [124]:
def keygeneration(n, party_i):
    pkey_list = []
    skey_list = []
    for i in range(n):
        if  i == party_i:
            pkey_list.append(0)
            skey_list.append(0)
        else: 
            pk, sk = nb.crypto_kx_keypair()
            pkey_list.append(pk)
            skey_list.append(sk)
    return  pkey_list,skey_list 


def keyexchange(n, party_i, my_pkey_list, my_skey_list, other_pkey_list):
    common_key_list = []
    for i in range(n):
        #Generate DH (common) keys 
        if i == party_i:
            common_key_list.append(0)
        else:
            if i > party_i:
                common_key_raw, _ = nb.crypto_kx_client_session_keys(my_pkey_list[i], my_skey_list[i], other_pkey_list[i])
            else:  
                _, common_key_raw = nb.crypto_kx_server_session_keys(my_pkey_list[i], my_skey_list[i], other_pkey_list[i])
            #Hash the common keys
            common_key = int.from_bytes(nb.crypto_hash_sha256(common_key_raw), byteorder='big')
            common_key_list.append(common_key)
    return common_key_list



def randomize(client, r, modulo, clientsign):
    for i in range(len(client)):
        # Call the double lenght pseudorsndom generator
        random.seed(r)
        rand            = random.getrandbits(256*2)
        rand_b_raw      = bin(rand)
        nr_zeros_append = 256 - (len(rand_b_raw) - 2)
        rand_b          = '0' * nr_zeros_append + rand_b_raw[2:]
        # Use first half to mask the inputs and second half as the next seed to the pseudorsndom generator
        R = int(rand_b[0:256], 2)
        r = int(rand_b[256:] , 2)
        
        client[i] = (client[i]+ (clientsign * R )) % modulo
    return r, client 


def randomize_all(party_i, common_key_list, client, modulo):
    
    for i in range(len(common_key_list)):
        if i == party_i:
             continue
        clientsign = 1 if i > party_i else -1
        common_key_list[i], client = randomize(client, common_key_list[i], modulo, clientsign)
        
    return common_key_list, client

In [125]:
n=2

t=[]
noiseS = [[t for i in range(n)] for j in range(n)]
nv=[]
noise = [[nv for i in range(n)] for j in range(n)]
nv1=[]
N = [[nv1 for i in range(n)] for j in range(n)]

#Generate public and secret key (sk,pk) pair per party

for party_i in range(n):
    
    pkey,skey = keygeneration(n,party_i)

    #Print the list of public keys to a Python object
    pk_file = open("pk_keys_" + str(party_i) + ".obj", 'wb') 
    pickle.dump(pkey, pk_file)
    pk_file.close()

    #Print the list of secret keys to a Python object
    sk_file = open("sk_keys_" + str(party_i) + ".obj", 'wb') 
    pickle.dump(skey, sk_file)
    sk_file.close()
    
    
    
# Open "pk_keys_i.obj" and store the secret keys
all_pkey_list = []
for i in range(n):
    file = open("pk_keys_" + str(i) + ".obj", 'rb') 
    all_pkey_list.append(pickle.load(file))
    file.close()


# Open "sk_keys_i.obj" and store the secret keys
all_skey_list = []
for i in range(n):
    file = open("sk_keys_" + str(i) + ".obj", 'rb') 
    all_skey_list.append(pickle.load(file))
    file.close()

for party_i in range(n):
    # Choose the keys for the corresponding party_i     
    other_pkey_list =  [all_pkey_list[i][party_i] for i in range(n)]
    pkey_list       = all_pkey_list[party_i]
    skey_list       = all_skey_list[party_i]

    #Call the function to generate the DH common keys
    common_key_list = keyexchange(n, party_i, pkey_list, skey_list, other_pkey_list)
    
    #generate DP noise vector (Gamma - Gamma) 
    
    data = pd.read_csv("logReg.csv")  
    X=data.iloc[:,:-1].values
    Lambda=1
    epsilon=1 # diff private loss parameter
    scale = 2/( len(X) * Lambda * epsilon)
    
    for j in range(n):          
            noiseS[party_i][j]=random.randint(1,1000)
    
    for j in range(n):
        a=np.random.gamma(1/n,scale, 1)
        b=np.random.gamma(1/n,scale, 1)
        noise[party_i][j]= b-a
        N[party_i][j]=noise[party_i][j]+noiseS[party_i][j]
        pickle.dump( N[party_i][j], open( "noise"+str(party_i)+str(j)+".p", "wb" ) ) 
   
    #Print common keys to a Python object
    file = open("Common_Keys_" + str(party_i) + ".obj", 'wb')
    for j in range(n):
        common_key_list[j]=common_key_list[j]/10e+77
    pickle.dump(common_key_list, file)

    file.close()



In [126]:
for i in range(n):
    
    total=0
    sSum=0
    noiseSum=0
    from ipynb.fs.full.LogisticRegression import logisticRegression
    w=logisticRegression()[1]
    file = open("Common_Keys_" + str(i) + ".obj", 'rb') 
    r=pickle.load(file)
    
    for k in range(n):
        if (i==k):
            total+=w
        elif(i>k):
            total+=(r[k]/10e+77)
        else:
            total-=(r[k]/10e+77)
    for k in range(n):
        sSum=sSum+noiseS[i][k]
    print(sSum)
    
    for l in range(n):
        noiseSum=noiseSum+N[l][i]
        
    total= total-sSum+noiseSum
    print(total)
    pickle.dump( total, open( "ro"+str(i)+".p", "wb" ) )
    

312
[-7.26847805e+74]
842
[7.26847805e+74]


In [128]:
#Server calculates weighted mean
    
sum=0      
for i in range(n):
    sum += pickle.load( open( "ro"+str(i)+".p", "rb" ) )
    
print(sum/n)

[0.]
